In [1]:
default_headers = {
          'Content-Type': 'application/json',
          'Access-Control-Allow-Origin': '*'
        }

In [2]:
from IPython.display import display_javascript, display_html, display
import uuid

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 150px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [3]:
import requests, json

In [4]:
engine = "http://jasper:8181"

In [5]:
e = requests.get(engine + "/jasper").json()

In [6]:
RenderJSON(e)

In [7]:
l = requests.get(engine + "/jasper/streams").json()

In [8]:
RenderJSON(l)

In [9]:
qs = requests.get(engine + "/jasper/queries").json()

In [10]:
RenderJSON(qs)

In [13]:
redstream = requests.get("http://redstream:1255/red")

In [14]:
RenderJSON(redstream.json())

In [75]:
body = {
    'id':'redcount',
    'tbox': 'file:///usr/src/wasp/colors.owl',
    'body': '''
        SELECT (COUNT(?r) as ?numRed)
        FROM NAMED WINDOW <rw> ON <http://streamhub:9292/streamhub/streams/red> [RANGE PT15S STEP PT5S]
        WHERE {
                WINDOW ?rw { ?r a <http://www.streamreasoning.org/ontologies/2018/9/colors#Red> .}
            }

    '''
}
    
data = json.dumps(body)

RenderJSON(body)

In [76]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [77]:
q.text

'{\n  "@id" : "queries/redcount",\n  "@type" : "vprov:Task",\n  "prov:generated" : {\n    "@id" : "streams/redcount"\n  },\n  "prov:uses" : {\n    "@id" : "http://streamhub:9292/streamhub/streams/red"\n  },\n  "@context" : {\n    "@base" : "http://jasper:8181/jasper/",\n    "rdf" : "http://www.w3.org/1999/02/22-rdf-syntax-ns#",\n    "vsd" : "http://w3id.org/rsp/vocals-sd#",\n    "vocals" : "http://w3id.org/rsp/vocals#",\n    "xsd" : "http://www.w3.org/2001/XMLSchema#",\n    "format" : "http://www.w3.org/ns/formats/",\n    "rdfs" : "http://www.w3.org/2000/01/rdf-schema#",\n    "vprov" : "http://w3id.org/rsp/vocals-prov#",\n    "dcat" : "http://www.w3.org/ns/dcat#",\n    "prov" : "http://www.w3.org/ns/prov#"\n  }\n}'

In [78]:
RenderJSON(q.json())

In [79]:
o = requests.post(engine + "/jasper/observers/redcount", data = json.dumps({'protocol':"HTTP", "retention":5}) , headers=default_headers).json()

In [80]:
os = requests.get(engine + "/jasper/observers/redcount").json()

In [81]:
RenderJSON(o)

In [82]:
RenderJSON(os)

In [43]:
a = requests.get("http://jasper:8182/jasper/streams/redcount/observers/349655421")

In [44]:
a

<Response [200]>

In [83]:
o = requests.post(engine + "/jasper/observers/redcount", data = json.dumps({'protocol':"WEBSOCKET"}) , headers=default_headers).json()

In [84]:
RenderJSON(o)

In [62]:
body = {
    'id':'redgreen',
    'tbox': 'file:///Users/riccardo/Desktop/colors.owl',
    'body': '''
    
SELECT (COUNT(?r) AS ?cntr) (count(?g) AS ?cntg) ((?cntr>?cntg) as ?moreRedsThanGreens)
FROM NAMED WINDOW <rw> ON <http://localhost:1255/red> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <gw> ON <http://localhost:2255/green> [RANGE PT15S STEP PT5S]
WHERE {
    { WINDOW ?rw { ?r a <http://www.streamreasoning.org/ontologies/2018/9/colors#Red> .} }
    UNION 
    { WINDOW ?gw { ?g a <http://www.streamreasoning.org/ontologies/2018/9/colors#Green> .} }
}

    '''
}
    
data = json.dumps(body)

RenderJSON(body)


In [63]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [64]:
q

<Response [400]>

In [58]:
RenderJSON(requests.get(engine + "/jasper/queries").json())

In [59]:
RenderJSON(q.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [61]:
o = requests.post(engine + "/jasper/observers/redgreen", data = json.dumps({'protocol':"HTTP", "retention":5}) , headers=default_headers).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [180]:
RenderJSON(o)

In [161]:
a = requests.get("http://docker.for.mac.localhost:8184/jasper/streams/redgreen/observers/764455011").json()
RenderJSON(a)

In [268]:
body = {
    'id':'yellow',
    'tbox': 'file:///Users/riccardo/Desktop/colors.owl',
    'body': '''
    
CONSTRUCT { 
           ?y a <http://www.streamreasoning.org/ontologies/2018/9/colors#Yellow>; 
               <http://www.streamreasoning.org/ontologies/2018/9/colors#from> ?r ; 
               <http://www.streamreasoning.org/ontologies/2018/9/colors#from> ?g .
}
FROM NAMED WINDOW <rw> ON <http://localhost:1255/red> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <gw> ON <http://localhost:2255/green> [RANGE PT15S STEP PT5S]
WHERE {
    WINDOW <rw> { ?r a <http://www.streamreasoning.org/ontologies/2018/9/colors#Red> .}

    WINDOW <gw> { ?g a <http://www.streamreasoning.org/ontologies/2018/9/colors#Green> .}

BIND( UUID() as ?y )

    '''
}
    
data = json.dumps(body)

RenderJSON(body)

In [269]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [235]:
q.text

'{\n  "@id" : "queries/yellow",\n  "@type" : "vprov:Task",\n  "prov:generated" : {\n    "@id" : "streams/yellow"\n  },\n  "prov:uses" : {\n    "@id" : "http://localhost:2255/green"\n  },\n  "@context" : {\n    "@base" : "http://localhost:8181/jasper/",\n    "rdf" : "http://www.w3.org/1999/02/22-rdf-syntax-ns#",\n    "vsd" : "http://w3id.org/rsp/vocals-sd#",\n    "vocals" : "http://w3id.org/rsp/vocals#",\n    "xsd" : "http://www.w3.org/2001/XMLSchema#",\n    "format" : "http://www.w3.org/ns/formats/",\n    "rdfs" : "http://www.w3.org/2000/01/rdf-schema#",\n    "vprov" : "http://w3id.org/rsp/vocals-prov#",\n    "dcat" : "http://www.w3.org/ns/dcat#",\n    "prov" : "http://www.w3.org/ns/prov#"\n  }\n}'

In [254]:
RenderJSON(q.json())

In [270]:
o = requests.post(engine + "/jasper/observers/yellow", data = json.dumps({'protocol':"WEBSOCKET"}) , headers=default_headers).json()

In [271]:
RenderJSON(o)

In [259]:
body = {
    'id':'reasoning1',
    'tbox': 'file:///Users/riccardo/Desktop/colors.owl',
    'body': '''
    
SELECT (COUNT(?w) AS ?cntWarm) (COUNT(?c) AS ?cntCold) ((?cntWarm>?cntCold) as ?moreWarmThanCold)  
FROM NAMED WINDOW <rw> ON <http://localhost:1255/red> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <gw> ON <http://localhost:2255/green> [RANGE PT15S STEP PT5S]
WHERE {
    { WINDOW <rw> { ?w a <http://www.streamreasoning.org/ontologies/2018/9/colors#Warm>.} }
    UNION
    { WINDOW <gw> { ?c a <http://www.streamreasoning.org/ontologies/2018/9/colors#Cold> .} }
}
    '''
}
    
data = json.dumps(body)

RenderJSON(body)

In [260]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [261]:
RenderJSON(q.json())

In [262]:
o = requests.post(engine + "/jasper/observers/reasoning1", data = json.dumps({'protocol':"WEBSOCKET"}) , headers=default_headers).json()

In [263]:
RenderJSON(o)

In [272]:
ss = requests.get(engine + "/jasper/streams", headers=default_headers).json()

In [273]:
RenderJSON(ss)

In [293]:
body = {
    'id':'reasoning2',
    'tbox': 'file:///Users/riccardo/Desktop/colors.owl',
    'body': '''
    
SELECT (COUNT(?p) AS ?cntPrimary) (COUNT(?d) AS ?cntDerived) (?cntPrimary > ?cntDerived AS ?morePrimaryThanDerived)
FROM NAMED WINDOW <rw> ON <http://localhost:1255/red> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <gw> ON <http://localhost:2255/green> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <bw> ON <http://localhost:3255/blue> [RANGE PT15S STEP PT5S]
FROM NAMED WINDOW <yw> ON <http://localhost:8181/jasper/streams/yellow> [RANGE PT15S STEP PT5S]
WHERE {
    { WINDOW ?pw { ?p a <http://www.streamreasoning.org/ontologies/2018/9/colors#Primary> . }}
    UNION
    { WINDOW ?dw { ?d a <http://www.streamreasoning.org/ontologies/2018/9/colors#Derived>. }}
}

    '''
}
    
data = json.dumps(body)

RenderJSON(body)

In [291]:
q = requests.post(engine + "/jasper/queries", data = data, headers=default_headers)

In [290]:
q.text

'{"error":"Duplicate Resource", "data": { "key": "{ "key": "http://localhost:8181/jasper/queries/reasoning2", "class": "String"}", "class": "Key"}}'

In [286]:
RenderJSON(q.json())

JSONDecodeError: Expecting ',' delimiter: line 1 column 53 (char 52)

In [277]:
o = requests.post(engine + "/jasper/observers/reasoning2", data = json.dumps({'protocol':"WEBSOCKET"}) , headers=default_headers).json()

In [278]:
RenderJSON(o)

In [335]:
qs = requests.get(engine + "/jasper/queries")

In [336]:
qs.json()

[]

In [429]:
requests.delete(engine + "/jasper/queries/redcount")

<Response [200]>